In [23]:
import pandas as pd
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
import xgboost

In [2]:
data_files = ['genderclassmodel.csv','gendermodel.csv',
'test.csv',
'train.csv']
for f in data_files:
    df = pd.read_csv(''.join(['data/',f]))
    print f
    print '--------------------------------'
    print df.head(2)
    print '--------------------------------'


genderclassmodel.csv
--------------------------------
   PassengerId  Survived
0          892         0
1          893         1
--------------------------------
gendermodel.csv
--------------------------------
   PassengerId  Survived
0          892         0
1          893         1
--------------------------------
test.csv
--------------------------------
   PassengerId  Pclass                              Name     Sex   Age  SibSp  \
0          892       3                  Kelly, Mr. James    male  34.5      0   
1          893       3  Wilkes, Mrs. James (Ellen Needs)  female  47.0      1   

   Parch  Ticket    Fare Cabin Embarked  
0      0  330911  7.8292   NaN        Q  
1      0  363272  7.0000   NaN        S  
--------------------------------
train.csv
--------------------------------
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   

                                                Name     Sex   Age  SibSp  \
0      

In [3]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
df_train.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [5]:
le = preprocessing.LabelEncoder()

categories_col =['Sex','Cabin','Embarked']

for col in categories_col:
    df_train[col+'_cat'] = le.fit_transform(df_train[col])
    

/home/ramon/anaconda2/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [6]:
df_train.columns

Index([u'PassengerId', u'Survived', u'Pclass', u'Name', u'Sex', u'Age',
       u'SibSp', u'Parch', u'Ticket', u'Fare', u'Cabin', u'Embarked',
       u'Sex_cat', u'Cabin_cat', u'Embarked_cat'],
      dtype='object')

In [7]:
def clean_df(df,survived=False):
    
    le = preprocessing.LabelEncoder()

    categories_col =['Sex','Cabin','Embarked']

    for col in categories_col:
        df[col+'_cat'] = le.fit_transform(df[col])
    
    cols_for_model = ['Pclass',  u'Age',
           u'SibSp', u'Parch', u'Fare',
           u'Sex_cat', u'Cabin_cat', u'Embarked_cat']
    
    if survived:
        cols_for_model.append('Survived')
    
    df_featured = df[cols_for_model]
    df_featured.describe()

    df_featured['Age'] = df_featured['Age'].fillna(df_featured.Age.mean())
    df_featured.describe()
    return df_featured


In [8]:
df_train_featured = clean_df(df_train,survived='Survived')

/home/ramon/anaconda2/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)
/home/ramon/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:

df_train_featured.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_cat,Cabin_cat,Embarked_cat,Survived
0,3,22.0,1,0,7.2500,1,0,3,0
1,1,38.0,1,0,71.2833,0,82,1,1
2,3,26.0,0,0,7.9250,0,0,3,1
3,1,35.0,1,0,53.1000,0,56,3,1
4,3,35.0,0,0,8.0500,1,0,3,0


In [10]:
df_test_featured = clean_df(df_test)

/home/ramon/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
clf = xgboost.XGBClassifier()

X, Y = df_train_featured.drop('Survived',1).values,df_train_featured.Survived.values


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)

In [13]:
clf.fit(X_train,y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [28]:
clf.predict(X_test)

array([0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0])

In [29]:
accuracy_score(clf.predict(X_test),y_test)

0.8044692737430168